In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
import random

In [2]:
def create_validation_split(train_dir, val_dir, val_split=0.2):
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    class_names = os.listdir(train_dir)

    for class_name in class_names:
        class_train_dir = os.path.join(train_dir, class_name)
        class_val_dir = os.path.join(val_dir, class_name)

        if not os.path.exists(class_val_dir):
            os.makedirs(class_val_dir)

        images = os.listdir(class_train_dir)
        num_val_images = int(len(images) * val_split)
        val_images = random.sample(images, num_val_images)

        for image in val_images:
            src_path = os.path.join(class_train_dir, image)
            dst_path = os.path.join(class_val_dir, image)
            shutil.move(src_path, dst_path)
        print(f"Moved {num_val_images} images from {class_train_dir} to {class_val_dir}")

# Example usage:
train_dir = 'train'
val_dir = 'validation'
create_validation_split(train_dir, val_dir, val_split=0.2)

Moved 52 images from train\Black-grass to validation\Black-grass
Moved 78 images from train\Charlock to validation\Charlock
Moved 57 images from train\Cleavers to validation\Cleavers
Moved 122 images from train\Common Chickweed to validation\Common Chickweed
Moved 44 images from train\Common wheat to validation\Common wheat
Moved 95 images from train\Fat Hen to validation\Fat Hen
Moved 130 images from train\Loose Silky-bent to validation\Loose Silky-bent
Moved 44 images from train\Maize to validation\Maize
Moved 103 images from train\Scentless Mayweed to validation\Scentless Mayweed
Moved 46 images from train\Shepherds Purse to validation\Shepherds Purse
Moved 99 images from train\Small-flowered Cranesbill to validation\Small-flowered Cranesbill
Moved 77 images from train\Sugar beet to validation\Sugar beet


In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

# Create validation dataset
validation_set = tf.keras.utils.image_dataset_from_directory(
    'validation',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

# Get the number of classes
class_names = training_set.class_names
num_classes = len(class_names)

Found 3805 files belonging to 12 classes.
Found 947 files belonging to 12 classes.


In [3]:
# Build the model
def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)

# Create an instance of the model
model = build_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=10
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 12)                  │           1,548 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 112,204 (438.30 KB)

 Trainable params: 111,756 (436.55 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 99s 801ms/step - accuracy: 0.4167 - loss: 1.7534 - val_accuracy: 0.2460 - val_loss: 2.8182
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 96s 803ms/step - accuracy: 0.6754 - loss: 0.9866 - val_accuracy: 0.4509 - val_loss: 1.5562
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 95s 798ms/step - accuracy: 0.7419 - loss: 0.7727 - val_accuracy: 0.6283 - val_loss: 1.0751
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 95s 795ms/step - accuracy: 0.7864 - loss: 0.6368 - val_accuracy: 0.6167 - val_loss: 1.1117
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 95s 795ms/step - accuracy: 0.8112 - loss: 0.5579 - val_accuracy: 0.7592 - val_loss: 0.6919
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 94s 793ms/step - accuracy: 0.8449 - loss: 0.4659 - val_accuracy: 0.6769 - val_loss: 0.9921
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 95s 794ms/step - accuracy: 0.8621 - loss: 0.3956 - val_accuracy: 0.6959 - val_loss: 0.9614
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 97s 811ms/step - accuracy: 0.8560 - loss: 0

In [4]:
#Training set Accuracy
train_loss, train_acc = model.evaluate(training_set)
print('Training accuracy:', train_acc*100)

119/119 ━━━━━━━━━━━━━━━━━━━━ 25s 211ms/step - accuracy: 0.6951 - loss: 0.8745
Training accuracy: 70.85413932800293


In [5]:
#Validation set Accuracy
val_loss, val_acc = model.evaluate(validation_set)
print('Validation accuracy:', val_acc*100)

30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 192ms/step - accuracy: 0.5986 - loss: 1.1013
Validation accuracy: 68.95459294319153
